# PyTao Basics

## Inititialize Tao

`Tao` is the basic object. Anything used to initialize Tao on the command line can be used to initialize a Tao object

In [ ]:
from pytao import Tao

In [ ]:
tao = Tao("-init $ACC_ROOT_DIR/bmad-doc/tao_examples/cesr/tao.init -noplot")

## Send a command

Anything that you would normally type at a Tao> prompt can be sent as a string. The return is a list of output strings. To send a command:

In [ ]:
tao.cmd("show lat 1:10")

Send a list of commands. This returns the corresponding list of outputs:

In [ ]:
tao.cmds(["set lattice model=design", "set ele Q00W x_offset = 1e-6"])

## Jupyter magic %%tao

This is an alternative way to send commands to Tao directly in the jupyter notebook, using the %%tao magic. Multiple lines can be executed.

In [ ]:
%%tao
sho lat 1:10
sho ele 4

## Interface commands

Output above from the `show` command is designed to be human-readable. In general you should **not** try to parse these strings for data. For data, Tao has a special set of commands to send back data suitable for parsing in Python (or other software). 

Below are the raw commands. 

In [ ]:
%%tao
help python

This data is returned as specially formatted lists

In [ ]:
tao.cmd("python orbit_at_s end")

Some commands have 'array_out' options. For example, this seems to return nothing:

In [ ]:
tao.cmd("python lat_list -array_out 1@0>>Q*|model orbit.floor.x")

But calling `.cmd_real` on the same command will get the data from an internal pointer:

In [ ]:
tao.cmd_real("python lat_list -array_out 1@0>>Q*|model orbit.floor.x")

# Tao method commands

For convenience, all of these commands are available as methods of the Tao class, and automatically parse the output.

For example, to get the orbit at an `s` position:

In [ ]:
tao.orbit_at_s(s_offset=1.2)

Some commands return arrays:

In [ ]:
tao.evaluate("data::cbar.11[1:10]|model")

## lat_list

`lat_list` can be used to efficiently extract array data. By default this returns an array of floats:

In [ ]:
s = tao.lat_list("*", "ele.s", verbose=True)
s[0:5]

These particulars keys will return integers:

In [ ]:
state = tao.lat_list("*", "orbit.state")
ix = tao.lat_list("*", "ele.ix_ele")
state.dtype, ix.dtype

And this one will return a list of strings:

In [ ]:
names = tao.lat_list("*", "ele.name")
names[0:5]

In [ ]:
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np

Simple plot of this data

In [ ]:
plt.plot(tao.lat_list("*", "ele.s"), tao.lat_list("*", "orbit.vec.1"), marker=".");

Be careful with the flags. The defaults are:

In [ ]:
plt.plot(tao.lat_list("*", "ele.s", flags="-array_out -track_only"));

`-index_order` will return

In [ ]:
plt.plot(tao.lat_list("*", "ele.s", flags="-array_out -index_order"));

# All method commands

There are many method commands. Please explore the documentation to find the ones that will best suit your needs.

In [ ]:
from pytao import interface_commands

all_cmds = [name for name in dir(Tao) if not name.startswith("_")]
for cmd in all_cmds:
    print(cmd)

There are many commands:

In [ ]:
len(all_cmds)

Each has documentation and an example associated with it:

In [ ]:
tao.data_d2?

# Other Tao instances

Unfortunately there can only be one Tao instance per process, because the internal structures are held in memory and accessed via ctypes. So this will replace the current Tao session in memory.

This looks like a new object:

In [ ]:
tao2 = Tao(
    "-init $ACC_ROOT_DIR/bmad-doc/tao_examples/csr_beam_tracking/tao.init -noplot"
)

but internally connects to:

In [ ]:
tao.lat_list("*", "ele.name")

## Bunch data

This example has bunch data. There are three methods:

- `tao.bunch_params` to get a dict of statistical data
    
- `tao.bunch1` to get coordinate data arrays
    
- `tao.bunch_data` to get a dict of many coordinate arrays. These can be used to instantiate a ParticleGroup object from the openPMD-beamphysics package.

Statistical data:

In [ ]:
stats = tao.bunch_params("end")
stats.keys()

Array data:

In [ ]:
x = tao.bunch1("end", coordinate="x")
px = tao.bunch1("end", coordinate="px")
plt.scatter(x, px);

The state will be returned as an integer array. 

In [ ]:
state = tao.bunch1("end", coordinate="state")
state.dtype

# ParticleGroup from openPMD-beamphysics

openPMD-beamphysics is an external package that can be useful for further bunch analysis, plotting, and conversion. 

https://github.com/ChristopherMayes/openPMD-beamphysics

Here is example usage to extract bunch data and instantiate as a `ParticleGroup` object.

*Note that the momentum units in openPMD-beamphysics are in eV/c, whereas Bmad's momenta are normalized by a refrence momentum.*

In [ ]:
from pmd_beamphysics import ParticleGroup

This data is suitable for the `ParticleGroup` class

In [ ]:
data = tao.bunch_data("end")
data.keys()

In [ ]:
P = ParticleGroup(data=data)

P.plot("x", "px")

Tao's write format is already in the openPMD-beamphysics, so particles can be written and read in this way as well.

In [ ]:
tao.cmd("write beam -at end test.h5")

In [ ]:
P2 = ParticleGroup("test.h5")
P2.plot("x", "px")

In [ ]:
# Cleanup
!rm test.h5

# Error handling and Debugging

All methods have a `raises=True` option. This will raise a RuntimeError if any errors are seen from Tao.


In [ ]:
tao.lat_list("*", "ele.s")

In [ ]:
try:
    tao.var("foobar")
except Exception as ex:
    print("Exception handled:", ex)

This suppresses the exceptions, returning the error text:

In [ ]:
tao.cmd("invalid_command", raises=False)

# Logging

All input commands are recorded as debug messages using standard Python logging. 

Enable stdout to see the log messages:

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

In [ ]:
tao.cmd("sho ele 2");

# Cleanup

In [ ]:
!rm csr_wake.dat